In [10]:
import sys
sys.path.append('../')
from time import sleep
from models.attention_series import Grade_regressor
import spectral
from configs.training_cfg import device
import torch
import ast
import numpy as np
from PIL import Image
import pandas as pd
from spectral import imshow
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt

spectral.settings.envi_support_nonlowercase_params = True

pre_list = []
erro_list = []

In [11]:
# 测试样本绝对误差
for k in range(1):
    model = Grade_regressor().to(device).eval()
    model.load_state_dict(torch.load("..\\ckpt\\粉末预训练_11000.pt".format(k+1)))
    result = []
    err = 0
    err_count = 0 
    pool = torch.nn.AvgPool2d(9,9)
    mask_rgb_values = [[255,242,0],[34,177,76],[255,0,88],[184,61,186]]

    spec_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]    # 选择评估的成像光谱图片编号


    for id in spec_id:
        img = spectral.envi.open("D:\\可见光粉末\\spectral_data\{}-RadianceConversion-CorrectFromMeasuredReference.bip.hdr".format(id))

        # 根据模型使用波段选择
        # img_data = torch.Tensor(img.asarray()/6000)[:,:,:-4]
        img_data = torch.Tensor(img.asarray()/6000)[:,:,:]
        mask = np.array(Image.open("D:\\可见光粉末\\spectral_data\\{}-RadianceConversion-CorrectFromMeasuredReference.bip.hdr_mask.png".format(id)))
        if mask.shape[2] == 4:
            mask = mask[:,:,:-1]
        gt_TFe = ast.literal_eval(img.metadata['gt_TFe'])
        gt = torch.Tensor(gt_TFe)
        gts = gt.__len__()

        with torch.no_grad():
            img_data = pool(img_data.permute(2,0,1)).permute(1,2,0)
            row,col,_ = img_data.shape
            heat_map = []

            for i in range(row):
                heat_map.append(model(img_data[i].to(device)).squeeze(1).unsqueeze(0).to("cpu"))   # 只评估Tfe
                torch.cuda.empty_cache()

        heat_map = torch.cat(heat_map, dim=0)

        predict_sum = torch.zeros(gts)
        pixel_count = torch.zeros(gts)/1e5

        for r in range(row):
            for c in range(col):
                for i in range(gts):
                    if mask[r*9+4,c*9+4].tolist() == mask_rgb_values[i]:
                        predict_sum[i] += heat_map[r,c] 
                        pixel_count[i] += 1

        prediction = predict_sum / pixel_count * 100

        err_list = ((prediction-gt)).tolist()
        pred_list = (prediction).tolist()
        for i in range(err_list.__len__()):
            erro_list.append(round(err_list[i],2))
            pre_list.append(round(pred_list[i], 2))
            

    # avg_err = err/(err_count)
    # print("{}k  avgmse:{}".format(k+1, avg_err))

IndexError: list index out of range

In [ ]:
prediction

tensor([30.0708, 34.7443])

In [ ]:
erro_list

[-1.51,
 1.46,
 -2.82,
 nan,
 -0.31,
 2.42,
 -1.47,
 nan,
 -1.17,
 -0.36,
 -1.53,
 nan,
 0.85,
 -3.14,
 -1.03,
 nan,
 0.05,
 3.58,
 -2.1,
 nan,
 0.44,
 1.06,
 -1.74,
 nan,
 -1.31,
 -2.95,
 4.55,
 nan,
 0.22,
 -2.02,
 -2.27,
 nan,
 -0.34,
 4.26,
 0.78,
 nan,
 7.21,
 -6.0,
 -1.45,
 nan,
 1.98,
 3.62,
 -15.12,
 nan,
 -2.57,
 0.99,
 -3.09,
 nan,
 2.75,
 -1.73,
 2.14,
 nan,
 2.98,
 10.09,
 3.15,
 nan,
 -2.08,
 -1.22,
 -2.71,
 nan,
 2.29,
 -3.67,
 -3.89,
 nan,
 -2.25,
 -2.47,
 2.51,
 nan,
 -0.1,
 -2.8,
 -0.89,
 nan,
 -0.27,
 -5.55,
 -5.12,
 nan,
 -1.92,
 -1.53,
 4.94,
 nan,
 -3.29,
 9.86,
 -5.7,
 nan,
 4.33,
 1.45,
 -4.15,
 nan,
 6.25,
 1.4,
 -2.58,
 nan,
 1.98,
 0.24,
 -3.01,
 nan,
 1.26,
 3.52,
 2.17,
 nan,
 -2.92,
 5.33,
 0.38,
 nan,
 1.37,
 5.71]

In [ ]:
pre_list

[23.16,
 16.81,
 29.84,
 nan,
 16.41,
 33.58,
 33.25,
 nan,
 28.78,
 23.65,
 22.62,
 nan,
 18.86,
 32.17,
 28.46,
 nan,
 11.45,
 30.09,
 14.11,
 nan,
 19.77,
 12.9,
 29.42,
 nan,
 25.54,
 32.05,
 23.69,
 nan,
 31.81,
 33.26,
 29.47,
 nan,
 28.73,
 33.47,
 27.13,
 nan,
 23.65,
 18.72,
 26.08,
 nan,
 26.06,
 27.48,
 50.58,
 nan,
 35.38,
 34.33,
 30.29,
 nan,
 26.35,
 23.01,
 25.75,
 nan,
 32.88,
 31.72,
 31.44,
 nan,
 29.17,
 29.89,
 27.46,
 nan,
 26.6,
 31.32,
 16.61,
 nan,
 18.96,
 30.3,
 19.31,
 nan,
 26.16,
 30.47,
 27.99,
 nan,
 22.65,
 29.04,
 25.67,
 nan,
 34.01,
 35.99,
 28.05,
 nan,
 30.02,
 29.54,
 28.43,
 nan,
 21.28,
 19.28,
 27.98,
 nan,
 28.4,
 26.04,
 17.97,
 nan,
 30.4,
 22.2,
 23.59,
 nan,
 22.73,
 24.89,
 25.98,
 nan,
 28.63,
 22.1,
 40.36,
 nan,
 30.07,
 34.74]